In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

In [ ]:
aiken = pd.read_csv('C://Users//jack//Documents//aiken_upload_pipeline//Python_scripts//AikenTempCSV//awb_db_complete11.04.2024.csv')
blancco = pd.read_csv('C://Users//jack//Documents//aiken_upload_pipeline//Python_scripts//exports//2024-04-12_merged_export.csv')

In [ ]:
display(aiken.head())
display(blancco.head())

In [ ]:
display(aiken.columns)
display(blancco.columns)

# save column lists as csv files
aiken_cols = pd.DataFrame(aiken.columns)
aiken_cols.to_csv('C://Users//jack//Documents//price_prediction_pipeline//csv//databasemergerTEST//aiken_cols.csv', index=False, header=False)

blancco_cols = pd.DataFrame(blancco.columns)
blancco_cols.to_csv('C://Users//jack//Documents//price_prediction_pipeline//csv//databasemergerTEST//blancco_cols.csv', index=False, header=False)

In [ ]:
blancco['CollectionID'] = blancco['Business name'].str.split('-').str[1]

# Drop columns that are not needed
blancco = blancco.drop(['Business name', 'BARCODE_y', 'Barcode_y'], axis=1)

blancco['Barcode'] = blancco['BARCODE_x'].combine_first(blancco['Barcode_x'])

blancco = blancco.drop(['BARCODE_x', 'Barcode_x'], axis=1)

display(blancco)

In [ ]:
# Separate multiple drive interface type entries into individual columns

df_split = blancco['Disk interface type'].str.split('/', expand=True)

df_split = df_split.iloc[:, :4]

df_split.columns = ['Disk1 type', 'Disk2 type', 'Disk3 type', 'Disk4 type']
df = blancco.join(df_split)

df_display = df.loc[:, 'Disk model':]

# Display the DataFrame
display(df_display)


In [ ]:

df_split = blancco['Disk model'].str.split('/', expand=True)

df_split = df_split.iloc[:, :4]

df_split.columns = ['Disk1 model', 'Disk2 model', 'Disk3 model', 'Disk4 model']
df = df.join(df_split)

df_display = df.loc[:, 'Disk model':]

# Display the DataFrame
display(df_display)


In [ ]:
df_split = blancco['Disk serial'].str.split('/', expand=True)

df_split = df_split.iloc[:, :4]

df_split.columns = ['Disk1 serial', 'Disk2 serial', 'Disk3 serial', 'Disk4 serial']
df = df.join(df_split)

df_display = df.loc[:, 'Disk model':]

# Display the DataFrame
display(df_display)


In [ ]:
df_split = blancco['Disk vendor'].str.split('/', expand=True)

df_split = df_split.iloc[:, :4]

df_split.columns = ['Disk1 vendor', 'Disk2 vendor', 'Disk3 vendor', 'Disk4 vendor']
df = df.join(df_split)

df_display = df.loc[:, 'Disk serial':]

# Display the DataFrame
display(df_display)


In [ ]:
df_split = df['Disk capacity'].str.split('/', expand=True)

df_split = df_split.iloc[:, :4]

df_split.columns = ['Disk1 capacity', 'Disk2 capacity', 'Disk3 capacity', 'Disk4 capacity']
blancco = df.join(df_split)

df_display = df.loc[:, 'Disk capacity':]

# Display the DataFrame
display(df_display)


In [ ]:
header_mapping = {
    'Barcode':'UnitID', 
    'CollectionID':'CollectionID', 
    'Erasure date':'Audited', 
    'System chassis type':'ProductType', 
    'System manufacturer':'Manufacturer',
    'System model':'Model', 
    'System serial':'SerialNumber', 
    'Display resolution':'Resolution', 
    'CPU model':'Processor',
    'CPU generation':'ProcGen', 
    'Memory capacity':'TotalRAM', 
    'Memory type':'RAM1Type', 
    'Disk1 capacity':'Storage1Size', 
    'Disk2 capacity':'Storage2Size', 
    'Disk3 capacity':'Storage3Size', 
    'Disk4 capacity':'Storage4Size',
    'Disk1 type':'Storage1Type',
    'Disk2 type':'Storage2Type',
    'Disk3 type':'Storage3Type',
    'Disk4 type':'Storage4Type',
    'Disk1 model':'Storage1Model', 
    'Disk2 model':'Storage2Model', 
    'Disk3 model':'Storage3Model', 
    'Disk4 model':'Storage4Model', 
    'Disk1 serial':'Storage1Serial', 
    'Disk2 serial':'Storage2Serial', 
    'Disk3 serial':'Storage3Serial', 
    'Disk4 serial':'Storage4Serial',
    'Disk1 vendor':'Storage1Manufacturer', 
    'Disk2 vendor':'Storage2Manufacturer', 
    'Disk3 vendor':'Storage3Manufacturer', 
    'Disk4 vendor':'Storage4Manufacturer', 
    'Battery model':'BatteryModel', 
    'Video card model':'GPU', 
    'Erasure technician':'User'
}

blancco = blancco.rename(columns=header_mapping)

cols_to_drop = ['Maximum CPU frequency', 'Video card vendor', 'Report verification', 'Erasure target model', 'Disk index', 'Disk ID', 'Disk capacity', 'Disk model', 'Disk serial', 'Disk vendor', 'Disk interface type']

blancco = blancco.drop(cols_to_drop, axis=1)

display(blancco)    


In [ ]:
# Add 'source' column to each DataFrame
aiken['source'] = 'aiken'
blancco['source'] = 'blancco'

# Concatenate the two DataFrames
df = pd.concat([aiken, blancco], ignore_index=True)

# Sort the DataFrame by date
df = df.sort_values(by='Audited')

# Reset the index
df = df.reset_index(drop=True)

# Lowercase all column headers
df.columns = map(str.lower, df.columns)

# Filter Nan values from 'collectionid' column
df = df[df['collectionid'].notna()]

# Drop rows where j is not in the collectionid column
df = df[df['collectionid'].str.contains('J')]

# Strip leading and trailing whitespace from collectionid column
df['collectionid'] = df['collectionid'].str.strip()

# Lowercase all values in all cells in the DataFrame
#df = df.apply(lambda x: x.astype(str).str.lower())

# Filter df by collectionID for string begining with 'j'
#df = df[df['collectionid'].str.startswith('j')]

# ensure that unitid and lotid are intergers
#df['unitid'] = df['unitid'].astype(int)

# Convert 'lotid_x' to float first, then to int to handle floating-point strings
df['lotid_x'] = df['lotid_x'].fillna(0).astype(float).astype(int)

# Display the rows which contain blancco in the source column
#df = df[df['source'].str.contains('blancco')]

display(df)

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('C://Users//jack//Documents//price_prediction_pipeline//csv//databasemergerTEST//stock_database.csv', index=False)